# 5.1.4 Aplicación en paralelo de transformadores usando FeatureUnion

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.svm import SVC

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append("../../../../") 

import utils.paths as path
from utils.paths2 import direcciones

ModuleNotFoundError: No module named 'utils'

- Permite unir una lista de transformadores en un nuevo transformador que combina sus salidas.
- Durante el proceso de entrenamiento, cada uno de los transformadores es entrenado independientemente.
- Los transformadores son aplicados en paralelo al set de datos, y las salidas son concatenadas lado a lado en una matriz.
- La combinación de FeatureUnion y Pipeline permite crear modelos complejos.

links: 

https://www.youtube.com/watch?v=F3J5-uTnhKQ

https://scikit-learn.org/stable/auto_examples/compose/plot_feature_union.html#sphx-glr-auto-examples-compose-plot-feature-union-py

In [ ]:
iris = load_iris()

X, y = iris.data, iris.target

#
# Crea un PCA que extrae unicamente dos (2) componentes
#
pca = PCA(n_components=2)

#
# Crea un transformador que selecciona únicamente la mejor característica
#
selection = SelectKBest(k=1)

#
# Crea la unión
#
combined_features = FeatureUnion(
    [
        ("pca", pca),
        ("univ_select", selection),
    ]
)

#
# Crea un nuevo vector de variables independientes que usando FeatureUnion
#
X_features = combined_features.fit(X, y).transform(X)
X_features.shape

(150, 3)

In [ ]:
#
# Crea el modelo de regresión
#
svm = SVC(kernel="linear")

#
# Crea un pipeline
#
pipeline = Pipeline(
    [
        ("features", combined_features),
        ("svm", svm),
    ]
)

#
# Búsqueda de la mejor combinación de hiperparámetros usando GridSearchCV
#
param_grid = dict(
    features__pca__n_components=[1, 2, 3],
    features__univ_select__k=[1, 2],
    svm__C=[0.1, 1, 10],
)

gridSearchCV = GridSearchCV(
    pipeline,
    param_grid=param_grid,
)

gridSearchCV.fit(X, y)

gridSearchCV.best_params_

{'features__pca__n_components': 3, 'features__univ_select__k': 1, 'svm__C': 10}

In [ ]:
print('ok_')